In [1]:
# 사용방법 - 복사하여 주석만 제거하여 사용
import sys
from pathlib import Path

# 현재 노트북(.ipynb)이 src/ 안에 있으니 상위 폴더(프로젝트 루트)를 추가
BASE_DIR    = Path().resolve().parent.parent    # Notebook이 src/ 안이라면 .parent
config_path = BASE_DIR / 'config' / 'data_paths.yaml'
sys.path.insert(0, str(BASE_DIR))


# 이제 바로 import
from scripts.data_loader import load_data

In [2]:
df_merged = load_data('Merged_Data_with_Season', section='processed')

In [3]:
df_merged

,Unnamed: 0,year_month,region,gender,age_group,visit_count,pm10,season
0,0,2006-01,가평,남자,1,74,NaN,겨울
1,1,2006-01,가평,남자,2,47,NaN,겨울
2,2,2006-01,가평,남자,3,11,NaN,겨울
3,3,2006-01,가평,남자,4,28,NaN,겨울
4,4,2006-01,가평,남자,5,55,NaN,겨울
...,...,...,...,...,...,...,...,...
370407,370407,2023-12,화천,남자,6,12,35.0,겨울
370408,370408,2023-12,화천,여자,1,5,35.0,겨울
370409,370409,2023-12,화천,여자,4,9,35.0,겨울
370410,370410,2023-12,화천,여자,5,12,35.0,겨울


In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer

df = df_merged.dropna(subset=['pm10'])
# Handle missing pm10: fill with median
df['pm10'] = df['pm10'].fillna(df['pm10'].median())

# Convert year_month to datetime and extract year/month
df['year_month'] = pd.to_datetime(df['year_month'], format="%Y-%m")
df['year'] = df['year_month'].dt.year
df['month'] = df['year_month'].dt.month

# Define features and target
X = df[['region', 'gender', 'age_group', 'pm10', 'season', 'year', 'month']]
y = df['visit_count']

# Define categorical and numerical features
categorical_features = ['region', 'gender', 'season', 'age_group']
numerical_features = ['pm10', 'year', 'month']

# Preprocessing for categorical and numerical data
preprocessor = ColumnTransformer(transformers=[
    ('cat', OneHotEncoder(handle_unknown='ignore'), categorical_features)
], remainder='passthrough')

# Build pipeline
model = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('regressor', RandomForestRegressor(random_state=42))
])

# Train/test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Train the model
model.fit(X_train, y_train)

# Predict
y_pred = model.predict(X_test)

# Evaluate
print("✅ RMSE:", mean_squared_error(y_test, y_pred, squared=False))
print("✅ R^2 Score:", r2_score(y_test, y_pred))


C:\Users\shinp\AppData\Local\Temp\ipykernel_40652\3291768873.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['pm10'] = df['pm10'].fillna(df['pm10'].median())
C:\Users\shinp\AppData\Local\Temp\ipykernel_40652\3291768873.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['year_month'] = pd.to_datetime(df['year_month'], format="%Y-%m")
C:\Users\shinp\AppData\Local\Temp\ipykernel_40652\3291768873.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try 